In [1]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


## **Importing the Required Libraries**

In [2]:
import pickle
import numpy as np
from scipy import signal
import skimage
from skimage.transform import resize
import matplotlib as mpl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import requests  # pip install requests
%matplotlib inline


import tensorflow as tf
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from scipy import stats
from IPython.display import display, HTML
from scipy import signal

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv3D,Conv2D, MaxPooling2D,TimeDistributed,LSTM,ConvLSTM2D
from keras.utils import np_utils


import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from matplotlib import colors
import cv2

In [ ]:
#Required Functions

Only chest sensor data

In [ ]:
#Function for Loading Data

In [3]:
# Loading data

def load_data(x):

  data_set = "E:\WESAD_DATASET\WESAD"
  sx_path = data_set + x
  with open(sx_path, 'rb') as file:
    sx_data = pickle.load(file, encoding='latin1')
  c_ax=sx_data['signal']['chest']['ACC'][0:,0]
  c_ay=sx_data['signal']['chest']['ACC'][0:,1]
  c_az=sx_data['signal']['chest']['ACC'][0:,2]
  c_ecg=sx_data['signal']['chest']['ECG'][:,0]
  c_emg=sx_data['signal']['chest']['EMG'][:,0]
  c_eda=sx_data['signal']['chest']['EDA'][:,0]
  c_temp=sx_data['signal']['chest']['Temp'][:,0]
  c_resp=sx_data['signal']['chest']['Resp'][:,0]
  w_label=sx_data['label']
  numpy_data=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
  numpy_data=numpy_data.T
  chest_df = pd.DataFrame(data=numpy_data,
                           columns=["c_ax", "c_ay", "c_az", "c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"])
  chest_df['w_label'] = chest_df['w_label'].astype('int')
  df = chest_df.loc[chest_df['w_label'].isin([1,2,3])]
  return df

In [19]:
#Function for creating windows

In [4]:
def window(data, size, shift):

  limit = 3500  # window size
  w = []   # windows
  w_count = 0 #no of windows
  j= 0
  for i in range(len(data)):
      res = data.iloc[j:size,:]
      if len(res.index) != limit:
         break
      else:
         res= res.T
         w.append(res)
         j = j + shift
         size = shift + size
         w_count = w_count + 1

  return w , w_count


In [5]:
# For making the data ready to be fed into the model

def data_ready(x_data , y_no, category):

  x = np.array(x_data)
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)

  return x,y

## **Function to generate spectogram images**

# **S2**

In [6]:
import os
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from PIL import Image
import cv2

def generate_spectrograms(data, path):
    """
    Generate spectrogram images from input data and save them in the specified path.

    Args:
    - data: numpy array of shape (num_examples, num_samples)
    - path: string representing path to save spectrogram images

    Returns:
    - None
    """
    os.makedirs(path, exist_ok=True)  # Create the directory if it doesn't exist

    for i in range(data.shape[0]):
        f, t, Sxx_i = signal.spectrogram(data[i, :])

        fig, ax = plt.subplots()
        ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')
        ax.set_axis_off()
        fig.savefig(os.path.join(path, f"spectrogram_{i+1}.png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig)


def preprocess_image(image_path, target_size):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to RGB if it has an alpha channel
    if image.mode == 'RGBA':
        image = image.convert('RGB')

    # Resize the image to the target size
    image = image.resize(target_size, Image.BICUBIC)

    # Convert the image to a numpy array
    image_array = np.array(image, dtype=np.float32)

    return image_array


def save_spectrograms_as_npy(path):
    """
    Load spectrogram images from the path, preprocess them, and save as a numpy array.

    Args:
    - path: string representing the path containing spectrogram images

    Returns:
    - None
    """
    filenames = sorted([f for f in os.listdir(path) if f.endswith('.png')])

    num_examples = len(filenames)
    Sxx = np.zeros((num_examples, 224, 224, 3), dtype=np.float32)

    for i, filename in enumerate(filenames):
        image_path = os.path.join(path, filename)

        # Preprocess the image
        preprocessed_image = preprocess_image(image_path, target_size=(224, 224))

        # Convert pixel values to the range of 0-255
        preprocessed_image = (preprocessed_image * 255).astype(np.uint8)

        # Assign the preprocessed image to Sxx
        Sxx[i] = preprocessed_image

        # Save the preprocessed image, replacing the original
        cv2.imwrite(image_path, cv2.cvtColor((preprocessed_image * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))

    np.save(os.path.join(path, 'Sxx.npy'), Sxx)
    

def concatenate_and_label_arrays(file_paths, save_path):
    arrays = []
    labels = []
    for i, file_path in enumerate(file_paths):
        array = np.load(file_path)
        arrays.append(array)
        num_samples = array.shape[0]
        label = np.zeros((num_samples, len(file_paths)))  # Create an array of zeros with shape (num_samples, len(file_paths))
        label[:, i] = 1  # Set the i-th column to 1 to indicate the label for this category
        labels.append(label)

    concatenated_array = np.concatenate(arrays)
    concatenated_labels = np.concatenate(labels)

    # Save concatenated array and concatenated labels as numpy arrays
    np.save(save_path + '/data.npy', concatenated_array)
    np.save(save_path + '/labels.npy', concatenated_labels)  

In [7]:
s2_path = "\S2\S2.pkl"

s2 = load_data(s2_path)
print(s2.head())
print()
print('Shape:')
print(s2.shape)
print()
print('dtypes:')
print(s2.dtypes)
print()
print('Value Counts:')
print(s2['w_label'].value_counts())
print()
s2_final = s2.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s2_final.head())

          c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
214583  0.8914 -0.1102 -0.2576  0.030945 -0.003708  5.710983  29.083618   
214584  0.8926 -0.1086 -0.2544  0.033646 -0.014145  5.719376  29.122437   
214585  0.8930 -0.1094 -0.2580  0.033005  0.010208  5.706406  29.115234   
214586  0.8934 -0.1082 -0.2538  0.031815  0.012634  5.712509  29.126709   
214587  0.8930 -0.1096 -0.2570  0.030350  0.002060  5.727005  29.100861   

          c_resp  w_label  
214583  1.191711        1  
214584  1.139832        1  
214585  1.141357        1  
214586  1.155090        1  
214587  1.133728        1  

Shape:
(1484700, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    800800
2    430500
3    253400
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
214583  0.030945 -0

In [8]:
print('Baseline:')
s2_base = s2_final.loc[s2_final['w_label'].isin([1])]
print(s2_base.shape)
s2_basex = s2_base.drop('w_label' ,axis = 1)
print(s2_basex.shape)
print()
print('Stress:')
s2_stress = s2_final.loc[s2_final['w_label'].isin([2])]
print(s2_stress.shape)
s2_stressx = s2_stress.drop('w_label',axis = 1)
print(s2_stressx.shape)
print()
print('Amusement:')
s2_amuse = s2_final.loc[s2_final['w_label'].isin([3])]
print(s2_amuse.shape)
s2_amusex = s2_amuse.drop('w_label',axis =1)
print(s2_amusex.shape)

Baseline:
(800800, 6)
(800800, 5)

Stress:
(430500, 6)
(430500, 5)

Amusement:
(253400, 6)
(253400, 5)


In [9]:
print('Creating windows:')
print()
print('Baseline:')
s2_basexw, count = window(s2_basex , 3500 ,1400)
print(count)
print(len(s2_basexw))

s2_basexwa , s2_basey  = data_ready(s2_basexw , count , 1)
print(s2_basexwa.shape)
print(s2_basey.shape)

print()
print('Stress:')
s2_stressxw, count = window(s2_stressx , 3500 ,1400)
print(count)
print(len(s2_stressxw))
s2_stressxwa , s2_stressy  = data_ready(s2_stressxw , count , 2)
print(s2_stressxwa.shape)
print(s2_stressy.shape)

print()
print('Amusement:')
s2_amusexw, count = window(s2_amusex , 3500 ,1400)
print(count)
print(len(s2_amusexw))
s2_amusexwa , s2_amusey  = data_ready(s2_amusexw , count , 3)
print(s2_amusexwa.shape)
print(s2_amusey.shape)

Creating windows:

Baseline:
570
570
(570, 5, 3500)
(570,)

Stress:
306
306
(306, 5, 3500)
(306,)

Amusement:
179
179
(179, 5, 3500)
(179,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


**S2_baseline**

In [10]:
# Load data
data = np.array(s2_basexwa[:, 0, :])
# Specify the paths to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S2_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S2_stress**

In [11]:
# Load data
data = np.array(s2_stressxwa[:, 0, :])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S2_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S2_amusement**

In [12]:
# Load data
data = np.array(s2_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S2_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S2**

In [125]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S2_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S2_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S2_amuse\Sxx.npy'
]

# Specify the path where to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S2_data_label"

# concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)


(1055, 224, 224, 3)
(1055, 3)


# **S3**

In [14]:
s3_path = "\S3\S3.pkl"
s3 = load_data(s3_path)
print(s3.head())
print()
print('Shape:')
print(s3.shape)
print()
print('dtypes:')
print(s3.dtypes)
print()
print('Value Counts:')
print(s3['w_label'].value_counts())
print()
s3_final = s3.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s3_final.head())


          c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
246998  0.7866 -0.0598  0.5048 -0.191849  0.058731  7.222748  32.989807   
246999  0.7910 -0.0616  0.5048 -0.186630  0.056259  7.224274  33.030365   
247000  0.7930 -0.0626  0.5018 -0.179260 -0.063034  7.222748  33.042389   
247001  0.7930 -0.0620  0.5022 -0.171432 -0.040787  7.221985  33.034882   
247002  0.7920 -0.0604  0.5038 -0.173264  0.116776  7.223892  33.045380   

           c_resp  w_label  
246998  12.303162        1  
246999  12.315369        1  
247000  12.339783        1  
247001  12.336731        1  
247002  12.318420        1  

Shape:
(1508500, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    798000
2    448000
3    262500
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp     c_resp  w_label
246998 -0.19

In [15]:
print('Baseline:')
s3_base = s3_final.loc[s3_final['w_label'].isin([1])]
print(s3_base.shape)
s3_basex = s3_base.drop('w_label' ,axis = 1)
print(s3_basex.shape)
print()
print('Stress:')
s3_stress = s3_final.loc[s3_final['w_label'].isin([2])]
print(s3_stress.shape)
s3_stressx = s3_stress.drop('w_label',axis = 1)
print(s3_stressx.shape)
print()
print('Amusement:')
s3_amuse = s3_final.loc[s3_final['w_label'].isin([3])]
print(s3_amuse.shape)
s3_amusex = s3_amuse.drop('w_label',axis =1)
print(s3_amusex.shape)

Baseline:
(798000, 6)
(798000, 5)

Stress:
(448000, 6)
(448000, 5)

Amusement:
(262500, 6)
(262500, 5)


In [16]:
print('Creating windows:')
print()
print('Baseline:')
s3_basexw, count = window(s3_basex , 3500 ,1400)
print(count)
print(len(s3_basexw))

s3_basexwa , s3_basey  = data_ready(s3_basexw , count , 1)
print(s3_basexwa.shape)
print(s3_basey.shape)

print()
print('Stress:')
s3_stressxw, count = window(s3_stressx , 3500 ,1400)
print(count)
print(len(s3_stressxw))
s3_stressxwa , s3_stressy  = data_ready(s3_stressxw , count , 2)
print(s3_stressxwa.shape)
print(s3_stressy.shape)

print()
print('Amusement:')
s3_amusexw, count = window(s3_amusex , 3500 ,1400)
print(count)
print(len(s3_amusexw))
s3_amusexwa , s3_amusey  = data_ready(s3_amusexw , count , 3)
print(s3_amusexwa.shape)
print(s3_amusey.shape)

Creating windows:

Baseline:
568
568
(568, 5, 3500)
(568,)

Stress:
318
318
(318, 5, 3500)
(318,)

Amusement:
186
186
(186, 5, 3500)
(186,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


### **S3_baseline**

In [17]:
# Load data
data = np.array(s3_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S3_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S3_stress**

In [18]:
# Load data
data = np.array(s3_stressxwa[:, 0, :])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S3_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S3_amusement**

In [19]:
# Load data
data = np.array(s3_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S3_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S3**

In [126]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S3_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S3_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S3_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S3_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1072, 224, 224, 3)
(1072, 3)


# **S4**

In [21]:
s4_path = "\S4\S4.pkl"
s4 = load_data(s4_path)
print(s4.head())
print()
print('Shape:')
print(s4.shape)
print()
print('dtypes:')
print(s4.dtypes)
print()
print('Value Counts:')
print(s4['w_label'].value_counts())
print()
s4_final = s4.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s4_final.head())

          c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
199705  0.6746 -0.2214 -0.6216  0.009567  0.011215  1.343536  32.681061   
199706  0.6742 -0.2198 -0.6236  0.003525  0.021652  1.316452  32.694519   
199707  0.6752 -0.2216 -0.6274 -0.000046  0.030212  1.325989  32.657135   
199708  0.6770 -0.2240 -0.6322  0.001968  0.012039  1.280975  32.691559   
199709  0.6760 -0.2230 -0.6282  0.001144  0.020920  1.357651  32.678101   

          c_resp  w_label  
199705 -2.406311        1  
199706 -2.383423        1  
199707 -2.360535        1  
199708 -2.345276        1  
199709 -2.320862        1  

Shape:
(1515501, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    810601
2    444500
3    260400
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
199705  0.009567  0

In [22]:
print('Baseline:')
s4_base = s4_final.loc[s4_final['w_label'].isin([1])]
print(s4_base.shape)
s4_basex = s4_base.drop('w_label' ,axis = 1)
print(s4_basex.shape)
print()
print('Stress:')
s4_stress = s4_final.loc[s4_final['w_label'].isin([2])]
print(s4_stress.shape)
s4_stressx = s4_stress.drop('w_label',axis = 1)
print(s4_stressx.shape)
print()
print('Amusement:')
s4_amuse = s4_final.loc[s4_final['w_label'].isin([3])]
print(s4_amuse.shape)
s4_amusex = s4_amuse.drop('w_label',axis =1)
print(s4_amusex.shape)

Baseline:
(810601, 6)
(810601, 5)

Stress:
(444500, 6)
(444500, 5)

Amusement:
(260400, 6)
(260400, 5)


In [23]:
print('Creating windows:')
print()
print('Baseline:')
s4_basexw, count = window(s4_basex , 3500 ,1400)
print(count)
print(len(s4_basexw))

s4_basexwa , s4_basey  = data_ready(s4_basexw , count , 1)
print(s4_basexwa.shape)
print(s4_basey.shape)

print()
print('Stress:')
s4_stressxw, count = window(s4_stressx , 3500 ,1400)
print(count)
print(len(s4_stressxw))
s4_stressxwa , s4_stressy  = data_ready(s4_stressxw , count , 2)
print(s4_stressxwa.shape)
print(s4_stressy.shape)

print()
print('Amusement:')
s4_amusexw, count = window(s4_amusex , 3500 ,1400)
print(count)
print(len(s4_amusexw))
s4_amusexwa , s4_amusey  = data_ready(s4_amusexw , count , 3)
print(s4_amusexwa.shape)
print(s4_amusey.shape)

Creating windows:

Baseline:
577
577
(577, 5, 3500)
(577,)

Stress:
316
316
(316, 5, 3500)
(316,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


### **S4_baseline**

In [24]:
# Load data
data = np.array(s4_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S4_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S4_stress**

In [25]:
# Load data
data = np.array(s4_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S4_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S4_amusement**

In [26]:
# Load data
data = np.array(s4_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S4_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S4**

In [27]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S4_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S4_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S4_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S4_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1077, 224, 224, 3)
(1077, 3)


# **S5**

In [28]:
s5_path = "\S5\S5.pkl"
s5 = load_data(s5_path)
print(s5.head())
print()
print('Shape:')
print(s5.shape)
print()
print('dtypes:')
print(s5.dtypes)
print()
print('Value Counts:')
print(s5['w_label'].value_counts())
print()
s5_final = s5.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s5_final.head())

          c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
195560  0.8408 -0.0390 -0.3862 -0.028976 -0.007828  6.387711  34.251160   
195561  0.8418 -0.0422 -0.3862 -0.029114 -0.003296  6.384277  34.252716   
195562  0.8386 -0.0390 -0.3906 -0.030212 -0.005264  6.398392  34.249634   
195563  0.8406 -0.0394 -0.3914 -0.030991 -0.003159  6.392288  34.298553   
195564  0.8424 -0.0434 -0.3914 -0.031906 -0.001968  6.385803  34.326050   

          c_resp  w_label  
195560  2.789307        1  
195561  2.891541        1  
195562  2.861023        1  
195563  2.833557        1  
195564  2.857971        1  

Shape:
(1551900, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    838600
2    451500
3    261800
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
195560 -0.028976 -0

In [29]:
print('Baseline:')
s5_base = s5_final.loc[s5_final['w_label'].isin([1])]
print(s5_base.shape)
s5_basex = s5_base.drop('w_label' ,axis = 1)
print(s5_basex.shape)
print()
print('Stress:')
s5_stress = s5_final.loc[s5_final['w_label'].isin([2])]
print(s5_stress.shape)
s5_stressx = s5_stress.drop('w_label',axis = 1)
print(s5_stressx.shape)
print()
print('Amusement:')
s5_amuse = s5_final.loc[s5_final['w_label'].isin([3])]
print(s5_amuse.shape)
s5_amusex = s5_amuse.drop('w_label',axis =1)
print(s5_amusex.shape)

Baseline:
(838600, 6)
(838600, 5)

Stress:
(451500, 6)
(451500, 5)

Amusement:
(261800, 6)
(261800, 5)


In [30]:
print('Creating windows:')
print()
print('Baseline:')
s5_basexw, count = window(s5_basex , 3500 ,1400)
print(count)
print(len(s5_basexw))

s5_basexwa , s5_basey  = data_ready(s5_basexw , count , 1)
print(s5_basexwa.shape)
print(s5_basey.shape)

print()
print('Stress:')
s5_stressxw, count = window(s5_stressx , 3500 ,1400)
print(count)
print(len(s5_stressxw))
s5_stressxwa , s5_stressy  = data_ready(s5_stressxw , count , 2)
print(s5_stressxwa.shape)
print(s5_stressy.shape)

print()
print('Amusement:')
s5_amusexw, count = window(s5_amusex , 3500 ,1400)
print(count)
print(len(s5_amusexw))
s5_amusexwa , s5_amusey  = data_ready(s5_amusexw , count , 3)
print(s5_amusexwa.shape)
print(s5_amusey.shape)

Creating windows:

Baseline:
597
597
(597, 5, 3500)
(597,)

Stress:
321
321
(321, 5, 3500)
(321,)

Amusement:
185
185
(185, 5, 3500)
(185,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


### **S5_baseline**

In [31]:
# Load data
data = np.array(s5_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S5_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S5_stress**

In [32]:
# Load data
data = np.array(s5_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S5_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S5_amusement**

In [33]:
# Load data
data = np.array(s5_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S5_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


In [34]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S5_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S5_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S5_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S5_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1103, 224, 224, 3)
(1103, 3)


# **S6**

In [35]:
s6_path = "\S6\S6.pkl"
s6 = load_data(s6_path)
print(s6.head())
print()
print('Shape:')
print(s6.shape)
print()
print('dtypes:')
print(s6.dtypes)
print()
print('Value Counts:')
print(s6['w_label'].value_counts())
print()
s6_final = s6.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s6_final.head())

          c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
449887  0.6280 -0.0082 -0.7272  0.016937  0.134720  9.645462  33.432587   
449888  0.6276 -0.0056 -0.7262  0.017853  0.216248  9.645462  33.459747   
449889  0.6270 -0.0054 -0.7200  0.016525  0.142136  9.646606  33.452240   
449890  0.6266 -0.0070 -0.7174  0.012863  0.033371  9.646225  33.453735   
449891  0.6242 -0.0078 -0.7202  0.007278 -0.024994  9.646988  33.426544   

          c_resp  w_label  
449887  4.010010        1  
449888  4.039001        1  
449889  3.993225        1  
449890  4.002380        1  
449891  3.996277        1  

Shape:
(1541400, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826000
2    455000
3    260400
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
449887  0.016937  0

In [36]:
print('Baseline:')
s6_base = s6_final.loc[s6_final['w_label'].isin([1])]
print(s6_base.shape)
s6_basex = s6_base.drop('w_label' ,axis = 1)
print(s6_basex.shape)
print()
print('Stress:')
s6_stress = s6_final.loc[s6_final['w_label'].isin([2])]
print(s6_stress.shape)
s6_stressx = s6_stress.drop('w_label',axis = 1)
print(s6_stressx.shape)
print()
print('Amusement:')
s6_amuse = s6_final.loc[s6_final['w_label'].isin([3])]
print(s6_amuse.shape)
s6_amusex = s6_amuse.drop('w_label',axis =1)
print(s6_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(455000, 6)
(455000, 5)

Amusement:
(260400, 6)
(260400, 5)


In [37]:
print('Creating windows:')
print()
print('Baseline:')
s6_basexw, count = window(s6_basex , 3500 ,1400)
print(count)
print(len(s6_basexw))

s6_basexwa , s6_basey  = data_ready(s6_basexw , count , 1)
print(s6_basexwa.shape)
print(s6_basey.shape)

print()
print('Stress:')
s6_stressxw, count = window(s6_stressx , 3500 ,1400)
print(count)
print(len(s6_stressxw))
s6_stressxwa , s6_stressy  = data_ready(s6_stressxw , count , 2)
print(s6_stressxwa.shape)
print(s6_stressy.shape)

print()
print('Amusement:')
s6_amusexw, count = window(s6_amusex , 3500 ,1400)
print(count)
print(len(s6_amusexw))
s6_amusexwa , s6_amusey  = data_ready(s6_amusexw , count , 3)
print(s6_amusexwa.shape)
print(s6_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
323
323
(323, 5, 3500)
(323,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


### **S6_baseline**

In [38]:
# Load data
data = np.array(s6_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S6_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S6_stress**

In [39]:
# Load data
data = np.array(s6_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S6_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S6_amusement**

In [43]:
# Load data
data = np.array(s6_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S6_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S6**

In [44]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S6_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S6_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S6_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S6_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1095, 224, 224, 3)
(1095, 3)


# **S7**

In [45]:
s7_path = "\S7\S7.pkl"
s7 = load_data(s7_path)
print(s7.head())
print()
print('Shape:')
print(s7.shape)
print()
print('dtypes:')
print(s7.dtypes)
print()
print('Value Counts:')
print(s7['w_label'].value_counts())
print()
s7_final = s7.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s7_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg      c_eda     c_temp  \
70442  0.9002 -0.0758 -0.2070 -0.013504 -0.031540  12.968826  34.202301   
70443  0.9018 -0.0744 -0.2090 -0.014694 -0.018723  12.967300  34.216064   
70444  0.8996 -0.0734 -0.2078 -0.014603 -0.012955  12.965775  34.217560   
70445  0.9014 -0.0750 -0.2088 -0.012955  0.005630  12.970352  34.222168   
70446  0.8964 -0.0726 -0.2074 -0.005539  0.014923  12.968826  34.197754   

         c_resp  w_label  
70442 -0.292969        1  
70443 -0.300598        1  
70444 -0.297546        1  
70445 -0.285339        1  
70446 -0.276184        1  

Shape:
(1538601, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    830200
2    448000
3    260401
Name: w_label, dtype: int64

          c_ecg     c_emg      c_eda     c_temp    c_resp  w_label
70442 -0.013504 -0.031540

In [46]:
print('Baseline:')
s7_base = s7_final.loc[s7_final['w_label'].isin([1])]
print(s7_base.shape)
s7_basex = s7_base.drop('w_label' ,axis = 1)
print(s7_basex.shape)
print()
print('Stress:')
s7_stress = s7_final.loc[s7_final['w_label'].isin([2])]
print(s7_stress.shape)
s7_stressx = s7_stress.drop('w_label',axis = 1)
print(s7_stressx.shape)
print()
print('Amusement:')
s7_amuse = s7_final.loc[s7_final['w_label'].isin([3])]
print(s7_amuse.shape)
s7_amusex = s7_amuse.drop('w_label',axis =1)
print(s7_amusex.shape)

Baseline:
(830200, 6)
(830200, 5)

Stress:
(448000, 6)
(448000, 5)

Amusement:
(260401, 6)
(260401, 5)


In [47]:
print('Creating windows:')
print()
print('Baseline:')
s7_basexw, count = window(s7_basex , 3500 ,1400)
print(count)
print(len(s7_basexw))

s7_basexwa , s7_basey  = data_ready(s7_basexw , count , 1)
print(s7_basexwa.shape)
print(s7_basey.shape)

print()
print('Stress:')
s7_stressxw, count = window(s7_stressx , 3500 ,1400)
print(count)
print(len(s7_stressxw))
s7_stressxwa , s7_stressy  = data_ready(s7_stressxw , count , 2)
print(s7_stressxwa.shape)
print(s7_stressy.shape)

print()
print('Amusement:')
s7_amusexw, count = window(s7_amusex , 3500 ,1400)
print(count)
print(len(s7_amusexw))
s7_amusexwa , s7_amusey  = data_ready(s7_amusexw , count , 3)
print(s7_amusexwa.shape)
print(s7_amusey.shape)

Creating windows:

Baseline:
591
591
(591, 5, 3500)
(591,)

Stress:
318
318
(318, 5, 3500)
(318,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


**S7_baseline**

In [48]:
# Load data
data = np.array(s7_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S7_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S7_stress**

In [49]:
# Load data
data = np.array(s7_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S7_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S7_amusement**

In [50]:
# Load data
data = np.array(s7_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S7_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S7**

In [51]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S7_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S7_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S7_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S7_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1093, 224, 224, 3)
(1093, 3)


# **S8**

In [52]:
s8_path = "\S8\S8.pkl"
s8 = load_data(s8_path)
print(s8.head())
print()
print('Shape:')
print(s8.shape)
print()
print('dtypes:')
print(s8.dtypes)
print()
print('Value Counts:')
print(s8['w_label'].value_counts())
print()
s8_final = s8.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s8_final.head())

          c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
110972  0.7112 -0.1910 -0.6150 -0.015884  0.005951  4.188919  34.315338   
110973  0.7112 -0.1934 -0.6150 -0.017349 -0.013412  4.185104  34.240479   
110974  0.7114 -0.1932 -0.6130 -0.014511 -0.014786  4.186630  34.301605   
110975  0.7108 -0.1922 -0.6122 -0.010391 -0.016663  4.201889  34.234375   
110976  0.7082 -0.1922 -0.6078 -0.011169 -0.002151  4.198837  34.300049   

          c_resp  w_label  
110972  0.465393        1  
110973  0.152588        1  
110974  0.471497        1  
110975  0.482178        1  
110976  0.506592        1  

Shape:
(1546299, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    818300
2    469000
3    258999
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
110972 -0.015884  0

In [53]:
print('Baseline:')
s8_base = s8_final.loc[s8_final['w_label'].isin([1])]
print(s8_base.shape)
s8_basex = s8_base.drop('w_label' ,axis = 1)
print(s8_basex.shape)
print()
print('Stress:')
s8_stress = s8_final.loc[s8_final['w_label'].isin([2])]
print(s8_stress.shape)
s8_stressx = s8_stress.drop('w_label',axis = 1)
print(s8_stressx.shape)
print()
print('Amusement:')
s8_amuse = s8_final.loc[s8_final['w_label'].isin([3])]
print(s8_amuse.shape)
s8_amusex = s8_amuse.drop('w_label',axis =1)
print(s8_amusex.shape)

Baseline:
(818300, 6)
(818300, 5)

Stress:
(469000, 6)
(469000, 5)

Amusement:
(258999, 6)
(258999, 5)


In [54]:
print('Creating windows:')
print()
print('Baseline:')
s8_basexw, count = window(s8_basex , 3500 ,1400)
print(count)
print(len(s8_basexw))

s8_basexwa , s8_basey  = data_ready(s8_basexw , count , 1)
print(s8_basexwa.shape)
print(s8_basey.shape)

print()
print('Stress:')
s8_stressxw, count = window(s8_stressx , 3500 ,1400)
print(count)
print(len(s8_stressxw))
s8_stressxwa , s8_stressy  = data_ready(s8_stressxw , count , 2)
print(s8_stressxwa.shape)
print(s8_stressy.shape)

print()
print('Amusement:')
s8_amusexw, count = window(s8_amusex , 3500 ,1400)
print(count)
print(len(s8_amusexw))
s8_amusexwa , s8_amusey  = data_ready(s8_amusexw , count , 3)
print(s8_amusexwa.shape)
print(s8_amusey.shape)

Creating windows:

Baseline:
583
583
(583, 5, 3500)
(583,)

Stress:
333
333
(333, 5, 3500)
(333,)

Amusement:
183
183
(183, 5, 3500)
(183,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


**S8_baseline**


In [55]:
# Load data
data = np.array(s8_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S8_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S8_stress**

In [56]:
# Load data
data = np.array(s8_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S8_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S8_amusement**

In [57]:
# Load data
data = np.array(s8_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S8_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S8**

In [58]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S8_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S8_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S8_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S8_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1099, 224, 224, 3)
(1099, 3)


# **S9**

In [59]:
s9_path = "\S9\S9.pkl"
s9 = load_data(s9_path)
print(s9.head())
print()
print('Shape:')
print(s9.shape)
print()
print('dtypes:')
print(s9.dtypes)
print()
print('Value Counts:')
print(s9['w_label'].value_counts())
print()
s9_final = s9.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s9_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
52672  0.9128 -0.0858 -0.0858  0.172073 -0.013321  2.141571  33.780762   
52673  0.9122 -0.0866 -0.0862  0.157242  0.006180  2.154922  33.774689   
52674  0.9140 -0.0890 -0.0838  0.143967  0.004440  2.141571  33.723114   
52675  0.9146 -0.0890 -0.0822  0.144882 -0.019180  2.159119  33.789917   
52676  0.9150 -0.0878 -0.0798  0.149963 -0.027008  2.142334  33.782318   

         c_resp  w_label  
52672  6.053162        1  
52673  6.065369        1  
52674  6.071472        1  
52675  6.065369        1  
52676  6.074524        1  

Shape:
(1537900, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826000
2    451500
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
52672  0.172073 -0.013321  2.141

In [60]:
print('Baseline:')
s9_base = s9_final.loc[s9_final['w_label'].isin([1])]
print(s9_base.shape)
s9_basex = s9_base.drop('w_label' ,axis = 1)
print(s9_basex.shape)
print()
print('Stress:')
s9_stress = s9_final.loc[s9_final['w_label'].isin([2])]
print(s9_stress.shape)
s9_stressx = s9_stress.drop('w_label',axis = 1)
print(s9_stressx.shape)
print()
print('Amusement:')
s9_amuse = s9_final.loc[s9_final['w_label'].isin([3])]
print(s9_amuse.shape)
s9_amusex = s9_amuse.drop('w_label',axis =1)
print(s9_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(451500, 6)
(451500, 5)

Amusement:
(260400, 6)
(260400, 5)


In [61]:
print('Creating windows:')
print()
print('Baseline:')
s9_basexw, count = window(s9_basex , 3500 ,1400)
print(count)
print(len(s9_basexw))

s9_basexwa , s9_basey  = data_ready(s9_basexw , count , 1)
print(s9_basexwa.shape)
print(s9_basey.shape)

print()
print('Stress:')
s9_stressxw, count = window(s9_stressx , 3500 ,1400)
print(count)
print(len(s9_stressxw))
s9_stressxwa , s9_stressy  = data_ready(s9_stressxw , count , 2)
print(s9_stressxwa.shape)
print(s9_stressy.shape)

print()
print('Amusement:')
s9_amusexw, count = window(s9_amusex , 3500 ,1400)
print(count)
print(len(s9_amusexw))
s9_amusexwa , s9_amusey  = data_ready(s9_amusexw , count , 3)
print(s9_amusexwa.shape)
print(s9_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
321
321
(321, 5, 3500)
(321,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


**S9_baseline**

In [62]:
# Load data
data = np.array(s9_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S9_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S9_stress**

In [63]:
# Load data
data = np.array(s9_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S9_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S9_amusement**

In [64]:
# Load data
data = np.array(s9_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S9_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S9**

In [65]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S9_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S9_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S9_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S9_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1093, 224, 224, 3)
(1093, 3)


# **S10**

In [66]:
s10_path = "\S10\S10.pkl"
s10 = load_data(s10_path)
print(s10.head())
print()
print('Shape:')
print(s10.shape)
print()
print('dtypes:')
print(s10.dtypes)
print()
print('Value Counts:')
print(s10['w_label'].value_counts())
print()
s10_final = s10.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s10_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
65000  0.8986  0.0874 -0.2018 -0.006775  0.013596  0.730515  33.891632   
65001  0.8994  0.0882 -0.2008 -0.002884  0.015198  0.745773  33.881012   
65002  0.8978  0.0870 -0.2022 -0.002289  0.016800  0.755310  33.873383   
65003  0.9000  0.0866 -0.1994  0.002792 -0.002243  0.722885  33.893158   
65004  0.9002  0.0870 -0.1990  0.008560 -0.007416  0.728989  33.900757   

         c_resp  w_label  
65000  0.685120        1  
65001  0.712585        1  
65002  0.737000        1  
65003  0.740051        1  
65004  0.733948        1  

Shape:
(1593900, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826000
2    507500
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
65000 -0.006775  0.013596  0.730

In [67]:
print('Baseline:')
s10_base = s10_final.loc[s10_final['w_label'].isin([1])]
print(s10_base.shape)
s10_basex = s10_base.drop('w_label' ,axis = 1)
print(s10_basex.shape)
print()
print('Stress:')
s10_stress = s10_final.loc[s10_final['w_label'].isin([2])]
print(s10_stress.shape)
s10_stressx = s10_stress.drop('w_label',axis = 1)
print(s10_stressx.shape)
print()
print('Amusement:')
s10_amuse = s10_final.loc[s10_final['w_label'].isin([3])]
print(s10_amuse.shape)
s10_amusex = s10_amuse.drop('w_label',axis =1)
print(s10_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(507500, 6)
(507500, 5)

Amusement:
(260400, 6)
(260400, 5)


In [68]:
print('Creating windows:')
print()
print('Baseline:')
s10_basexw, count = window(s10_basex , 3500 ,1400)
print(count)
print(len(s10_basexw))

s10_basexwa , s10_basey  = data_ready(s10_basexw , count , 1)
print(s10_basexwa.shape)
print(s10_basey.shape)

print()
print('Stress:')
s10_stressxw, count = window(s10_stressx , 3500 ,1400)
print(count)
print(len(s10_stressxw))
s10_stressxwa , s10_stressy  = data_ready(s10_stressxw , count , 2)
print(s10_stressxwa.shape)
print(s10_stressy.shape)

print()
print('Amusement:')
s10_amusexw, count = window(s10_amusex , 3500 ,1400)
print(count)
print(len(s10_amusexw))
s10_amusexwa , s10_amusey  = data_ready(s10_amusexw , count , 3)
print(s10_amusexwa.shape)
print(s10_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
361
361
(361, 5, 3500)
(361,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


**S10_baseline**

In [69]:
# Load data
data = np.array(s10_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S10_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S10_stress**


In [70]:
# Load data
data = np.array(s10_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S10_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**S10_amusement**

In [71]:
# Load data
data = np.array(s10_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S10_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S10**

In [72]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S10_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S10_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S10_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S10_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1133, 224, 224, 3)
(1133, 3)


# **S11**

In [73]:
s11_path = "\S11\S11.pkl"
s11 = load_data(s11_path)
print(s11.head())
print()
print('Shape:')
print(s11.shape)
print()
print('dtypes:')
print(s11.dtypes)
print()
print('Value Counts:')
print(s11['w_label'].value_counts())
print()
s11_final = s11.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s11_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
59937  0.8968  0.0414 -0.1698 -0.017395  0.002014  6.644821  33.677673   
59938  0.8978  0.0418 -0.1714 -0.012772  0.010117  6.648254  33.612549   
59939  0.9044  0.0406 -0.1708 -0.007599  0.003891  6.637192  33.665527   
59940  0.9000  0.0414 -0.1766 -0.010574 -0.011810  6.637573  33.657959   
59941  0.8978  0.0408 -0.1698 -0.015152 -0.024765  6.638718  33.665527   

         c_resp  w_label  
59937  1.760864        1  
59938  1.785278        1  
59939  1.788330        1  
59940  1.777649        1  
59941  1.817322        1  

Shape:
(1559600, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826000
2    476000
3    257600
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
59937 -0.017395  0.002014  6.644

In [74]:
print('Baseline:')
s11_base = s11_final.loc[s11_final['w_label'].isin([1])]
print(s11_base.shape)
s11_basex = s11_base.drop('w_label' ,axis = 1)
print(s11_basex.shape)
print()
print('Stress:')
s11_stress = s11_final.loc[s11_final['w_label'].isin([2])]
print(s11_stress.shape)
s11_stressx = s11_stress.drop('w_label',axis = 1)
print(s11_stressx.shape)
print()
print('Amusement:')
s11_amuse = s11_final.loc[s11_final['w_label'].isin([3])]
print(s11_amuse.shape)
s11_amusex = s11_amuse.drop('w_label',axis =1)
print(s11_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(476000, 6)
(476000, 5)

Amusement:
(257600, 6)
(257600, 5)


In [75]:
print('Creating windows:')
print()
print('Baseline:')
s11_basexw, count = window(s11_basex , 3500 ,1400)
print(count)
print(len(s11_basexw))

s11_basexwa , s11_basey  = data_ready(s11_basexw , count , 1)
print(s11_basexwa.shape)
print(s11_basey.shape)

print()
print('Stress:')
s11_stressxw, count = window(s11_stressx , 3500 ,1400)
print(count)
print(len(s11_stressxw))
s11_stressxwa , s11_stressy  = data_ready(s11_stressxw , count , 2)
print(s11_stressxwa.shape)
print(s11_stressy.shape)

print()
print('Amusement:')
s11_amusexw, count = window(s11_amusex , 3500 ,1400)
print(count)
print(len(s11_amusexw))
s11_amusexwa , s11_amusey  = data_ready(s11_amusexw , count , 3)
print(s11_amusexwa.shape)
print(s11_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
338
338
(338, 5, 3500)
(338,)

Amusement:
182
182
(182, 5, 3500)
(182,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


# **S11_baseline**

In [76]:
# Load data
data = np.array(s11_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S11_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S11_stress**

In [77]:
# Load data
data = np.array(s11_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S11_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S11_amusement**

In [78]:
# Load data
data = np.array(s11_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S11_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S11**

In [79]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S11_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S11_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S11_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S11_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1108, 224, 224, 3)
(1108, 3)


# **S13**

In [80]:
s13_path = "\S13\S13.pkl"
s13 = load_data(s13_path)
print(s13.head())
print()
print('Shape:')
print(s13.shape)
print()
print('dtypes:')
print(s13.dtypes)
print()
print('Value Counts:')
print(s13['w_label'].value_counts())
print()
s13_final = s13.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s13_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
61534  0.8978 -0.0228  0.0586 -0.206268 -0.003983  2.383423  32.814331   
61535  0.9018 -0.0222  0.0564 -0.237213  0.040970  2.416992  32.830811   
61536  0.9042 -0.0226  0.0568 -0.269028  0.076492  2.375793  32.794861   
61537  0.9078 -0.0246  0.0606 -0.287247  0.037903  2.373123  32.832306   
61538  0.9106 -0.0298  0.0570 -0.296310  0.020004  2.376175  32.806824   

         c_resp  w_label  
61534  0.721741        1  
61535  0.750732        1  
61536  0.776672        1  
61537  0.762939        1  
61538  0.787354        1  

Shape:
(1558201, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826001
2    464800
3    267400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
61534 -0.206268 -0.003983  2.383

In [81]:
print('Baseline:')
s13_base = s13_final.loc[s13_final['w_label'].isin([1])]
print(s13_base.shape)
s13_basex = s13_base.drop('w_label' ,axis = 1)
print(s13_basex.shape)
print()
print('Stress:')
s13_stress = s13_final.loc[s13_final['w_label'].isin([2])]
print(s13_stress.shape)
s13_stressx = s13_stress.drop('w_label',axis = 1)
print(s13_stressx.shape)
print()
print('Amusement:')
s13_amuse = s13_final.loc[s13_final['w_label'].isin([3])]
print(s13_amuse.shape)
s13_amusex = s13_amuse.drop('w_label',axis =1)
print(s13_amusex.shape)

Baseline:
(826001, 6)
(826001, 5)

Stress:
(464800, 6)
(464800, 5)

Amusement:
(267400, 6)
(267400, 5)


In [82]:
print('Creating windows:')
print()
print('Baseline:')
s13_basexw, count = window(s13_basex , 3500 ,1400)
print(count)
print(len(s13_basexw))

s13_basexwa , s13_basey  = data_ready(s13_basexw , count , 1)
print(s13_basexwa.shape)
print(s13_basey.shape)

print()
print('Stress:')
s13_stressxw, count = window(s13_stressx , 3500 ,1400)
print(count)
print(len(s13_stressxw))
s13_stressxwa , s13_stressy  = data_ready(s13_stressxw , count , 2)
print(s13_stressxwa.shape)
print(s13_stressy.shape)

print()
print('Amusement:')
s13_amusexw, count = window(s13_amusex , 3500 ,1400)
print(count)
print(len(s13_amusexw))
s13_amusexwa , s13_amusey  = data_ready(s13_amusexw , count , 3)
print(s13_amusexwa.shape)
print(s13_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
330
330
(330, 5, 3500)
(330,)

Amusement:
189
189
(189, 5, 3500)
(189,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


## **S13_baseline**

In [83]:
# Load data
data = np.array(s13_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S13_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S13_stress**

In [84]:
# Load data
data = np.array(s13_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S13_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S13_amusement**

In [85]:
# Load data
data = np.array(s13_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S13_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S13**

In [86]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S13_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S13_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S13_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S13_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1107, 224, 224, 3)
(1107, 3)


# **S14**

In [87]:
s14_path = "\S14\S14.pkl"
s14 = load_data(s14_path)
print(s14.head())
print()
print('Shape:')
print(s14.shape)
print()
print('dtypes:')
print(s14.dtypes)
print()
print('Value Counts:')
print(s14['w_label'].value_counts())
print()
s14_final = s14.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s14_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
18337  0.7342 -0.0674 -0.5618  1.499222 -0.011673  2.809906  32.851776   
18338  0.7350 -0.0688 -0.5622  1.499222 -0.015060  2.825165  32.929749   
18339  0.7330 -0.0662 -0.5638  1.499084 -0.002609  2.797699  32.845764   
18340  0.7348 -0.0686 -0.5646  1.498947 -0.002106  2.817535  32.862244   
18341  0.7354 -0.0696 -0.5642  1.499130 -0.009247  2.827072  32.866760   

         c_resp  w_label  
18337  2.146912        1  
18338  1.878357        1  
18339  2.174377        1  
18340  2.165222        1  
18341  2.195740        1  

Shape:
(1558901, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826000
2    472500
3    260401
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
18337  1.499222 -0.011673  2.809

In [88]:
print('Baseline:')
s14_base = s14_final.loc[s14_final['w_label'].isin([1])]
print(s14_base.shape)
s14_basex = s14_base.drop('w_label' ,axis = 1)
print(s14_basex.shape)
print()
print('Stress:')
s14_stress = s14_final.loc[s14_final['w_label'].isin([2])]
print(s14_stress.shape)
s14_stressx = s14_stress.drop('w_label',axis = 1)
print(s14_stressx.shape)
print()
print('Amusement:')
s14_amuse = s14_final.loc[s14_final['w_label'].isin([3])]
print(s14_amuse.shape)
s14_amusex = s14_amuse.drop('w_label',axis =1)
print(s14_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(472500, 6)
(472500, 5)

Amusement:
(260401, 6)
(260401, 5)


In [89]:
print('Creating windows:')
print()
print('Baseline:')
s14_basexw, count = window(s14_basex , 3500 ,1400)
print(count)
print(len(s14_basexw))

s14_basexwa , s14_basey  = data_ready(s14_basexw , count , 1)
print(s14_basexwa.shape)
print(s14_basey.shape)

print()
print('Stress:')
s14_stressxw, count = window(s14_stressx , 3500 ,1400)
print(count)
print(len(s14_stressxw))
s14_stressxwa , s14_stressy  = data_ready(s14_stressxw , count , 2)
print(s14_stressxwa.shape)
print(s14_stressy.shape)

print()
print('Amusement:')
s14_amusexw, count = window(s14_amusex , 3500 ,1400)
print(count)
print(len(s14_amusexw))
s14_amusexwa , s14_amusey  = data_ready(s14_amusexw , count , 3)
print(s14_amusexwa.shape)
print(s14_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
336
336
(336, 5, 3500)
(336,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


## **S14_baseline**

In [90]:
# Load data
data = np.array(s14_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S14_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S14_stress**

In [91]:
# Load data
data = np.array(s14_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S14_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S14_amusement**

In [92]:
# Load data
data = np.array(s14_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S14_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S14**

In [94]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S14_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S14_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S14_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S14_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1108, 224, 224, 3)
(1108, 3)


# **S15**

In [95]:
s15_path = "\S15\S15.pkl"
s15 = load_data(s15_path)
print(s15.head())
print()
print('Shape:')
print(s15.shape)
print()
print('dtypes:')
print(s15.dtypes)
print()
print('Value Counts:')
print(s15['w_label'].value_counts())
print()
s15_final = s15.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s15_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
77161  0.9086  0.0728 -0.1022 -0.178162 -0.026321  1.413727  33.535339   
77162  0.9110  0.0762 -0.1038 -0.180222 -0.026459  1.419830  33.535339   
77163  0.9102  0.0738 -0.1030 -0.183334 -0.018860  1.416016  33.532318   
77164  0.9090  0.0748 -0.1010 -0.187088 -0.015106  1.424026  33.536865   
77165  0.9106  0.0744 -0.1014 -0.186768 -0.016159  1.426697  33.541412   

         c_resp  w_label  
77161 -2.075195        1  
77162 -2.064514        1  
77163 -2.098083        1  
77164 -2.069092        1  
77165 -2.067566        1  

Shape:
(1563100, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    822500
2    480200
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
77161 -0.178162 -0.026321  1.413

In [96]:
print('Baseline:')
s15_base = s15_final.loc[s15_final['w_label'].isin([1])]
print(s15_base.shape)
s15_basex = s15_base.drop('w_label' ,axis = 1)
print(s15_basex.shape)
print()
print('Stress:')
s15_stress = s15_final.loc[s15_final['w_label'].isin([2])]
print(s15_stress.shape)
s15_stressx = s15_stress.drop('w_label',axis = 1)
print(s15_stressx.shape)
print()
print('Amusement:')
s15_amuse = s15_final.loc[s15_final['w_label'].isin([3])]
print(s15_amuse.shape)
s15_amusex = s15_amuse.drop('w_label',axis =1)
print(s15_amusex.shape)

Baseline:
(822500, 6)
(822500, 5)

Stress:
(480200, 6)
(480200, 5)

Amusement:
(260400, 6)
(260400, 5)


In [97]:
print('Creating windows:')
print()
print('Baseline:')
s15_basexw, count = window(s15_basex , 3500 ,1400)
print(count)
print(len(s15_basexw))

s15_basexwa , s15_basey  = data_ready(s15_basexw , count , 1)
print(s15_basexwa.shape)
print(s15_basey.shape)

print()
print('Stress:')
s15_stressxw, count = window(s15_stressx , 3500 ,1400)
print(count)
print(len(s15_stressxw))
s15_stressxwa , s15_stressy  = data_ready(s15_stressxw , count , 2)
print(s15_stressxwa.shape)
print(s15_stressy.shape)

print()
print('Amusement:')
s15_amusexw, count = window(s15_amusex , 3500 ,1400)
print(count)
print(len(s15_amusexw))
s15_amusexwa , s15_amusey  = data_ready(s15_amusexw , count , 3)
print(s15_amusexwa.shape)
print(s15_amusey.shape)

Creating windows:

Baseline:
586
586
(586, 5, 3500)
(586,)

Stress:
341
341
(341, 5, 3500)
(341,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


## **S15_baseline**

In [98]:
# Load data
data = np.array(s15_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S15_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S15_stress**

In [99]:
# Load data
data = np.array(s15_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S15_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S15_amusement**

In [100]:
# Load data
data = np.array(s15_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S15_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S15**

In [101]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S15_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S15_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S15_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S15_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1111, 224, 224, 3)
(1111, 3)


# **S16**

In [102]:
s16_path = "\S16\S16.pkl"
s16 = load_data(s16_path)
print(s16.head())
print()
print('Shape:')
print(s16.shape)
print()
print('dtypes:')
print(s16.dtypes)
print()
print('Value Counts:')
print(s16['w_label'].value_counts())
print()
s16_final = s16.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s16_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
68859  0.9022  0.0442 -0.0654  0.030533  0.016342  0.988770  33.376709   
68860  0.9048  0.0438 -0.0648  0.044449 -0.008194  0.972366  33.363129   
68861  0.9070  0.0418 -0.0626  0.050903 -0.026413  0.966263  33.376709   
68862  0.9072  0.0390 -0.0594  0.051590  0.002747  1.010132  33.379761   
68863  0.9054  0.0408 -0.0578  0.052917  0.003113  0.963593  33.331451   

         c_resp  w_label  
68859  6.391907        1  
68860  6.391907        1  
68861  6.404114        1  
68862  6.398010        1  
68863  6.410217        1  

Shape:
(1554701, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826000
2    471101
3    257600
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
68859  0.030533  0.016342  0.988

In [103]:
print('Baseline:')
s16_base = s16_final.loc[s16_final['w_label'].isin([1])]
print(s16_base.shape)
s16_basex = s16_base.drop('w_label' ,axis = 1)
print(s16_basex.shape)
print()
print('Stress:')
s16_stress = s16_final.loc[s16_final['w_label'].isin([2])]
print(s16_stress.shape)
s16_stressx = s16_stress.drop('w_label',axis = 1)
print(s16_stressx.shape)
print()
print('Amusement:')
s16_amuse = s16_final.loc[s16_final['w_label'].isin([3])]
print(s16_amuse.shape)
s16_amusex = s16_amuse.drop('w_label',axis =1)
print(s16_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(471101, 6)
(471101, 5)

Amusement:
(257600, 6)
(257600, 5)


In [104]:
print('Creating windows:')
print()
print('Baseline:')
s16_basexw, count = window(s16_basex , 3500 ,1400)
print(count)
print(len(s16_basexw))

s16_basexwa , s16_basey  = data_ready(s16_basexw , count , 1)
print(s16_basexwa.shape)
print(s16_basey.shape)

print()
print('Stress:')
s16_stressxw, count = window(s16_stressx , 3500 ,1400)
print(count)
print(len(s16_stressxw))
s16_stressxwa , s16_stressy  = data_ready(s16_stressxw , count , 2)
print(s16_stressxwa.shape)
print(s16_stressy.shape)

print()
print('Amusement:')
s16_amusexw, count = window(s16_amusex , 3500 ,1400)
print(count)
print(len(s16_amusexw))
s16_amusexwa , s16_amusey  = data_ready(s16_amusexw , count , 3)
print(s16_amusexwa.shape)
print(s16_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
335
335
(335, 5, 3500)
(335,)

Amusement:
182
182
(182, 5, 3500)
(182,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


## **S16_baseline**

In [105]:
# Load data
data = np.array(s16_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S16_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S16_stress**

In [106]:
# Load data
data = np.array(s16_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S16_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


## **S16_amusement**

In [107]:
# Load data
data = np.array(s16_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S16_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


**Labels_S16**

In [108]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S16_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S16_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S16_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S16_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1105, 224, 224, 3)
(1105, 3)


# **S17**

In [115]:
s17_path = "\S17\S17.pkl"
s17 = load_data(s17_path)
print(s17.head())
print()
print('Shape:')
print(s17.shape)
print()
print('dtypes:')
print(s17.dtypes)
print()
print('Value Counts:')
print(s17['w_label'].value_counts())
print()
s17_final = s17.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s17_final.head())

         c_ax    c_ay    c_az     c_ecg     c_emg     c_eda     c_temp  \
64564  0.7838 -0.4066 -0.3706 -0.014648 -0.001968  5.543137  32.567444   
64565  0.7842 -0.4040 -0.3678 -0.001236 -0.005539  5.542755  32.567444   
64566  0.7864 -0.4056 -0.3688  0.007278  0.017761  5.556107  32.561493   
64567  0.7876 -0.4078 -0.3682  0.009201  0.032181  5.558014  32.559998   
64568  0.7870 -0.4066 -0.3726  0.007141 -0.011032  5.564117  32.558502   

         c_resp  w_label  
64564  2.757263        1  
64565  2.742004        1  
64566  2.603149        1  
64567  2.749634        1  
64568  2.760315        1  

Shape:
(1593200, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int32
dtype: object

Value Counts:
1    826700
2    506100
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
64564 -0.014648 -0.001968  5.543

In [116]:
print('Baseline:')
s17_base = s17_final.loc[s17_final['w_label'].isin([1])]
print(s17_base.shape)
s17_basex = s17_base.drop('w_label' ,axis = 1)
print(s17_basex.shape)
print()
print('Stress:')
s17_stress = s17_final.loc[s17_final['w_label'].isin([2])]
print(s17_stress.shape)
s17_stressx = s17_stress.drop('w_label',axis = 1)
print(s17_stressx.shape)
print()
print('Amusement:')
s17_amuse = s17_final.loc[s17_final['w_label'].isin([3])]
print(s17_amuse.shape)
s17_amusex = s17_amuse.drop('w_label',axis =1)
print(s17_amusex.shape)

Baseline:
(826700, 6)
(826700, 5)

Stress:
(506100, 6)
(506100, 5)

Amusement:
(260400, 6)
(260400, 5)


In [117]:
print('Creating windows:')
print()
print('Baseline:')
s17_basexw, count = window(s17_basex , 3500 ,1400)
print(count)
print(len(s17_basexw))

s17_basexwa , s17_basey  = data_ready(s17_basexw , count , 1)
print(s17_basexwa.shape)
print(s17_basey.shape)

print()
print('Stress:')
s17_stressxw, count = window(s17_stressx , 3500 ,1400)
print(count)
print(len(s17_stressxw))
s17_stressxwa , s17_stressy  = data_ready(s17_stressxw , count , 2)
print(s17_stressxwa.shape)
print(s17_stressy.shape)

print()
print('Amusement:')
s17_amusexw, count = window(s17_amusex , 3500 ,1400)
print(count)
print(len(s17_amusexw))
s17_amusexwa , s17_amusey  = data_ready(s17_amusexw , count , 3)
print(s17_amusexwa.shape)
print(s17_amusey.shape)

Creating windows:

Baseline:
589
589
(589, 5, 3500)
(589,)

Stress:
360
360
(360, 5, 3500)
(360,)

Amusement:
184
184
(184, 5, 3500)
(184,)


C:\Users\user\AppData\Local\Temp/ipykernel_19452/3869773005.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)


### **S17_baseline**

In [118]:
# Load data
data = np.array(s17_basexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S17_baseline'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S17_stress**

In [119]:
# Load data
data = np.array(s17_stressxwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S17_stress'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **S17_amusement**

In [120]:
# Load data
data = np.array(s17_amusexwa[:,0,:])
# Specify the path to save the spectrogram images
path = 'D:\WESAD_code\WESAD_final\S17_amuse'
# Generate spectrogram images and save them as .png files
Sxx = generate_spectrograms(data, path)
save_spectrograms_as_npy(path)

C:\Users\user\AppData\Local\Temp/ipykernel_19452/3137945625.py:25: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ax.pcolormesh(t, f, Sxx_i, norm=colors.LogNorm(vmin=Sxx_i.min(), vmax=Sxx_i.max()), cmap='inferno')


### **Labels_s17**

In [121]:
# Provide the file paths for participant 1
file_paths = [
    'D:\WESAD_code\WESAD_final\S17_baseline\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S17_stress\Sxx.npy',
    'D:\WESAD_code\WESAD_final\S17_amuse\Sxx.npy'
]

# Specify the path where you want to save the concatenated arrays and labels
save_path = "D:\WESAD_code\WESAD_final\S17_data_label"

# Call the concatenate_and_label_arrays function
concatenate_and_label_arrays(file_paths, save_path)

concatenated_array = np.load(save_path + '/data.npy')
concatenated_labels = np.load(save_path + '/labels.npy')

# Access the shape of the concatenated array and concatenated labels
print(concatenated_array.shape)
print(concatenated_labels.shape)

(1133, 224, 224, 3)
(1133, 3)


### **Concatenate data from all the subject arrays**

In [122]:
#Concatenate data from all the subject arrays
folder_paths = [
    "D:\WESAD_code\WESAD_final\S2_data_label", 
    "D:\WESAD_code\WESAD_final\S3_data_label", 
    "D:\WESAD_code\WESAD_final\S4_data_label", 
    "D:\WESAD_code\WESAD_final\S5_data_label", 
    "D:\WESAD_code\WESAD_final\S6_data_label", 
    "D:\WESAD_code\WESAD_final\S7_data_label", 
    "D:\WESAD_code\WESAD_final\S8_data_label", 
    "D:\WESAD_code\WESAD_final\S9_data_label", 
    "D:\WESAD_code\WESAD_final\S10_data_label", 
    "D:\WESAD_code\WESAD_final\S11_data_label", 
    "D:\WESAD_code\WESAD_final\S13_data_label", 
    "D:\WESAD_code\WESAD_final\S14_data_label", 
    "D:\WESAD_code\WESAD_final\S15_data_label", 
    "D:\WESAD_code\WESAD_final\S16_data_label",
    "D:\WESAD_code\WESAD_final\S17_data_label"
]

arrays = []
for folder_path in folder_paths:
    file_path = os.path.join(folder_path, "data.npy")
    array = np.load(file_path)
    arrays.append(array)
data_array = np.concatenate(arrays, axis=0)
save_path = "D:\WESAD_code\WESAD_final\Subject_array\data_array"
np.save(save_path, data_array)
print(data_array.shape)

(16492, 224, 224, 3)


### **Concatenate labels from all the subjects**

In [123]:
#Concatenate labels from all the subjects
folder_paths = [
    "D:\WESAD_code\WESAD_final\S2_data_label", 
    "D:\WESAD_code\WESAD_final\S3_data_label", 
    "D:\WESAD_code\WESAD_final\S4_data_label", 
    "D:\WESAD_code\WESAD_final\S5_data_label", 
    "D:\WESAD_code\WESAD_final\S6_data_label", 
    "D:\WESAD_code\WESAD_final\S7_data_label", 
    "D:\WESAD_code\WESAD_final\S8_data_label", 
    "D:\WESAD_code\WESAD_final\S9_data_label", 
    "D:\WESAD_code\WESAD_final\S10_data_label", 
    "D:\WESAD_code\WESAD_final\S11_data_label", 
    "D:\WESAD_code\WESAD_final\S13_data_label", 
    "D:\WESAD_code\WESAD_final\S14_data_label", 
    "D:\WESAD_code\WESAD_final\S15_data_label", 
    "D:\WESAD_code\WESAD_final\S16_data_label",
    "D:\WESAD_code\WESAD_final\S17_data_label"
]

arrays = []
for folder_path in folder_paths:
    file_path = os.path.join(folder_path, "labels.npy")
    array = np.load(file_path)
    arrays.append(array)
label_array = np.concatenate(arrays, axis=0)
save_path = "D:\WESAD_code\WESAD_final\Labels_array\label_array"
np.save(save_path, label_array)
print(label_array.shape)

(16492, 3)


### **Split the dataset**

In [124]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming the dataset is stored in the variable `data` and the corresponding labels in `labels`

# Splitting the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(data_array, label_array, test_size=0.2, random_state=42)

# Checking the shape of each set
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

Training data shape: (13193, 224, 224, 3)
Training labels shape: (13193, 3)
Testing data shape: (3299, 224, 224, 3)
Testing labels shape: (3299, 3)
